# Fitting concave penalties with a two stage estimator via the LLA algorithm

Following Fan et al. 2014 we fit concave penalties such as SCAD with a two stag algorithm.

1. Find a reasonable initial guess (e.g. with a Lasso estimate).

2. Refine the initial guess using the local linear approximation (LLA) algorithm

If our initializer is "good enough" then this procedure comes with strong statistical guarantee after **one step** of the LLA algorithm. 


In [1]:
import numpy as np
from ya_glm.toy_data import sample_sparse_lin_reg

from ya_glm.backends.fista.LinearRegression import FcpLLA, FcpLLACV, LassoCV

In [2]:
# toy sparse linear regression data
X, y = sample_sparse_lin_reg(n_samples=100, n_features=10, n_nonzero=5,
                             random_state=234)[0:2]

# Single concave estimator

In [3]:
FcpLLA.__init__

<function ya_glm.pen_glms.GlmFcpLLA.GlmFcpLLA.__init__(self, pen_val=1, pen_func='scad', pen_func_kws={}, init='default', lla_n_steps=1, lla_kws={}, ridge_pen_val=None, ridge_weights=None, tikhonov=None, groups=None, fit_intercept=True, standardize=False, opt_kws={})>

### Specifying the initializer

In [4]:
# you can specify the initial estimator
init = LassoCV() # either unfitted
# init = LassoCV().fit(X, y) # or already fitted
# init = 'default' # FcpLLA comes with a sensible default estimator  
# init = {'intercept': 0,
#         'coef': np.zeros(X.shape[1])} # or you can manually specify the initial values

est = FcpLLA(init=init).fit(X, y)

In [5]:
# est = FcpLLA(lla_n_steps=1) # by defualt we do one LLA step
est = FcpLLA(lla_n_steps=1000) # but you can run the LLA algorithm to convergence

est.fit(X, y)

FcpLLA(lla_n_steps=1000)

# Cross-validation

In [6]:
est = FcpLLA(init=LassoCV())

# note we fit the init esimator only once before running cross-validation
# each fold is initialized from the same place!
est_cv = FcpLLACV(estimator=est).fit(X, y)

In [7]:
# for concave penalties fit with the LLA algorithm we 
# can also compute the largest reasonable penalty value above which
# the algorithm returns 0
est = FcpLLA(init=LassoCV().fit(X, y))
pen_val_max = est.get_pen_val_max(X, y, init_data=est.get_init_data(X, y))

est.set_params(pen_val=pen_val_max)
est.fit(X, y)
est.coef_

array([ 0.00000000e+00,  0.00000000e+00, -1.36847748e-07,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00])